## EDA using Million Playlist Database in SQLite
**Author:** Jamilla Akhund-Zade

###Introduction:
The Million Playlist Data (MPD) in its raw format is made up of CSVs 0 through 999, each with 1000 playlists. The raw MPD was merged into a single CSV file called 'combined_playlist_v2.csv'. This CSV file has an extra row denoting what CSV file the playlist came from.

The columns of the MPD are as follows:

1. **pid:** playlist ID within the original CSV
2. **pos:** position of song within the playlist
3. **artist_name:** name of the artist (string)
4. **track_uri:** unique track Spotify Identifier
5. **artist_uri:** unique artist Spotify Identifier
6. **track_name:** name of the track (string)
7. **album_uri:** unique album Spotify Identifier
8. **duration_ms:** track duration in ms
9. **album_name:** name of album
10. **file_name:** identifier of CSV where playlists came from
11. **pidfile_name:** unique playlist identifier

The MPD is 12GB, which is too large to hold in memory, so I will create an SQLite database so that I can do EDA efficiently, while the data is held out-of-memory. I will use the MacOS built-in SQLite command-line tools:

```
> sqlite3
sqlite> .open mpd.db #open brand-new database
sqlite> CREATE TABLE mpd(
    pid INTEGER,
    pos INTEGER,
    artist_name TEXT,
    track_uri TEXT,
    artist_uri TEXT,
    track_name TEXT,
    album_uri TEXT,
    duration_ms INTEGER,
    album_name TEXT,
    file_name TEXT,
    pidfile_name TEXT
);
sqlite> .mode csv #set mode to csv
sqlite> .import combined_playlist_v2.csv mpd #import csv data to table mpd

#create artist table
CREATE TABLE artists (
   id INTEGER NOT NULL PRIMARY KEY,
   artist_uri TEXT NOT NULL,
   artist_name TEXT NOT NULL
);
INSERT INTO artists(artist_uri, artist_name) SELECT DISTINCT artist_uri, artist_name FROM mpd;

#create album table
CREATE TABLE albums (
   id INTEGER NOT NULL PRIMARY KEY,
   album_uri TEXT NOT NULL,
   album_name TEXT NOT NULL
);
INSERT INTO albums(album_uri, album_name) SELECT DISTINCT album_uri, album_name FROM mpd;

#create song table
CREATE TABLE songs (
   track_id INTEGER NOT NULL PRIMARY KEY,
   track_uri TEXT NOT NULL,
   track_name TEXT NOT NULL,
   duration_ms INTEGER NOT NULL,
   artist_uri TEXT NOT NULL,
   album_uri TEXT NOT NULL
);
INSERT INTO songs(track_uri, track_name, duration_ms, artist_uri, album_uri) 
SELECT DISTINCT track_uri, track_name, duration_ms, artist_uri, album_uri FROM mpd;

ALTER TABLE songs RENAME TO old_songs;

CREATE TABLE songs AS
SELECT track_id, track_uri, track_name, duration_ms, artists.id AS artist_id, albums.id AS album_id
FROM old_songs
LEFT JOIN artists ON old_songs.artist_uri = artists.artist_uri
LEFT JOIN albums ON old_songs.album_uri = albums.album_uri;

DROP TABLE old_songs;

PRAGMA foreign_keys = ON;
ALTER TABLE songs RENAME TO old_songs;
CREATE TABLE songs (
   track_id INTEGER NOT NULL PRIMARY KEY,
   track_uri TEXT NOT NULL,
   track_name TEXT NOT NULL,
   duration_ms INTEGER NOT NULL,
   artist_id INTEGER NOT NULL,
   album_id INTEGER NOT NULL,
   FOREIGN KEY (artist_id) REFERENCES artists(id),
   FOREIGN KEY (album_id) REFERENCES albums(id)  
);
INSERT INTO songs
SELECT * FROM old_songs;

DROP TABLE old_songs;

PRAGMA foreign_keys = OFF;

#create playlist table
ALTER TABLE mpd RENAME TO old_mpd;

CREATE TABLE mpd AS
SELECT pid, pos, songs.track_id AS track_id, file_name, pidfile_name
FROM old_mpd
LEFT JOIN songs ON old_mpd.track_uri = songs.track_uri;

ALTER TABLE mpd RENAME TO temp_mpd;
CREATE TABLE mpd (
   id INTEGER NOT NULL PRIMARY KEY,
   pid INTEGER NOT NULL,
   pos INTEGER NOT NULL,
   track_id INTEGER NOT NULL,
   file_name TEXT NOT NULL,
   pidfile_name TEXT NOT NULL,
   FOREIGN KEY (track_id) REFERENCES songs(track_id)
);
INSERT INTO mpd(pid, pos, track_id, file_name, pidfile_name)
SELECT pid, pos, track_id, file_name, pidfile_name FROM temp_mpd;

DROP TABLE temp_mpd, old_mpd;

PRAGMA foreign_keys = OFF;
```

In [1]:
#load libraries
import pandas as pd
import sqlite3

In [2]:
#connect to sqlite database
con = sqlite3.connect("/Users/jamillaakhund-zade/CS109A/Spotify_Project/data/mpd.db")

In [6]:
#count number of unique records
cur = con.cursor() #cursor to hold query results

cur.execute('SELECT MAX(id) FROM mpd') 
#note: this only works accurately since mpd is a static table, where rows are not inserted/deleted
print(f'Total records: {cur.fetchone()[0]}')

Total records: 66346428


In [8]:
#count distinct songs
cur = con.cursor() #cursor to hold query results

cur.execute('SELECT MAX(track_id) FROM songs')
print(f'unique songs: {cur.fetchone()[0]}')

unique songs: 2262292
